In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE193442"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE193442"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE193442.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE193442.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE193442.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: PBMC'], 1: ['cell type: KIR+ CD8 T']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable

# Check gene expression data availability
# Based on the Series title and description, this appears to be a transcriptional profiling dataset
# This suggests gene expression data is likely available
is_gene_available = True

# Analyze clinical feature availability
# From the Sample Characteristics Dictionary, we don't have explicit trait (Multiple_sclerosis), age, or gender information
# The data only shows tissue (PBMC) and cell type (KIR+ CD8 T) information

# Set availability of trait, age, and gender
trait_row = None  # No explicit trait information available
age_row = None    # No age information available
gender_row = None # No gender information available

# Define conversion functions (these won't be used but defined for completeness)
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() in ['ms', 'multiple sclerosis']:
        return 1
    elif value.lower() in ['control', 'healthy', 'normal']:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    value = value.lower()
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    return None

# Initial validation of dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# No need to extract clinical features since trait data is not available (trait_row is None)


False

### Step 3: Gene Data Extraction

In [4]:
# The SuperSeries nature of GSE193442 is causing issues with our standard data extraction
# Let's try to check if we can find any SubSeries information

import gzip
import re

def extract_subseries_info(soft_file_path):
    """Extract SubSeries information from a SuperSeries SOFT file"""
    subseries_ids = []
    
    try:
        with gzip.open(soft_file_path, 'rt') as f:
            for line in f:
                if line.startswith('!Series_relation'):
                    # Look for SubSeries relation entries
                    match = re.search(r'SubSeries of:(\S+)', line)
                    if match:
                        subseries_ids.append(match.group(1))
                    # Also check for "SuperSeries of" pattern which lists the component series
                    match = re.search(r'SuperSeries of:(\S+)', line)
                    if match:
                        subseries_ids.append(match.group(1))
    except Exception as e:
        print(f"Error reading SOFT file: {e}")
    
    return subseries_ids

# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Check if we can find subseries information
print("Checking for SubSeries information in the SuperSeries...")
subseries = extract_subseries_info(soft_file)
print(f"SubSeries found: {subseries}")

# 3. Try direct extraction method with additional debugging
print("\nAttempting direct extraction with debugging:")
try:
    # Modified approach to print more information about the file
    with gzip.open(matrix_file, 'rt') as file:
        # Print first 10 lines to understand file structure
        print("First 10 lines of the matrix file:")
        for i, line in enumerate(file):
            if i < 10:
                print(f"Line {i+1}: {line.strip()}")
            if i == 0 and "SuperSeries" in line:
                print("This confirms the file is a SuperSeries metadata file")
            if "!series_matrix_table_begin" in line:
                print(f"Found table marker at line {i+1}")
                break
        else:
            print("No table marker found in the file")
            
    # Try standard extraction again but with error handling
    try:
        gene_data = get_genetic_data(matrix_file)
        print("\nGene data extraction stats:")
        print(f"Number of rows: {gene_data.shape[0]}")
        print(f"Number of columns: {gene_data.shape[1]}")
        
        if gene_data.shape[0] > 0:
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        else:
            print("No gene data rows found. This confirms this is a SuperSeries without direct gene expression data.")
    except Exception as e:
        print(f"Error during gene data extraction: {e}")
        
except Exception as e:
    print(f"Error examining matrix file: {e}")

# 4. Update data availability flag based on our findings
is_gene_available = False  # Updating based on our inspection
print(f"\nUpdated gene data availability: {is_gene_available}")

# 5. Re-validate cohort info with updated gene availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

Checking for SubSeries information in the SuperSeries...
SubSeries found: []

Attempting direct extraction with debugging:
First 10 lines of the matrix file:
Line 1: !Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
Line 2: !Series_geo_accession	"GSE193442"
Line 3: !Series_status	"Public on Mar 08 2022"
Line 4: !Series_submission_date	"Jan 11 2022"
Line 5: !Series_last_update_date	"Apr 20 2022"
Line 6: !Series_pubmed_id	"35258337"
Line 7: !Series_summary	"This SuperSeries is composed of the SubSeries listed below."
Line 8: !Series_overall_design	"Refer to individual Series"
Line 9: !Series_type	"Expression profiling by high throughput sequencing"
Line 10: !Series_type	"Other"
Found table marker at line 69



Gene data extraction stats:
Number of rows: 0
Number of columns: 4512
No gene data rows found. This confirms this is a SuperSeries without direct gene expression data.

Updated gene data availability: False


False